# Защита персональных данных клиентов

Необходимо защитить данные клиентов страховой компании «Хоть потоп». Разработать такой метод преобразования данных, чтобы по ним было сложно восстановить персональную информацию. Также обосновать корректность его работы. 

Нужно защитить данные, чтобы при преобразовании качество моделей машинного обучения не ухудшилось. Подбирать наилучшую модель не требуется.

<h1>Содержание<span class="tocSkip"></span></h1>
<div class="toc"><ul class="toc-item"><li><span><a href="#Загрузка-данных" data-toc-modified-id="Загрузка-данных-1"><span class="toc-item-num">1&nbsp;&nbsp;</span>Загрузка данных</a></span></li><li><span><a href="#Умножение-матриц" data-toc-modified-id="Умножение-матриц-2"><span class="toc-item-num">2&nbsp;&nbsp;</span>Умножение матриц</a></span></li><li><span><a href="#Алгоритм-преобразования" data-toc-modified-id="Алгоритм-преобразования-3"><span class="toc-item-num">3&nbsp;&nbsp;</span>Алгоритм преобразования</a></span><ul class="toc-item"><li><span><a href="#Предложите-алгоритм-преобразования-данных-для-решения-задачи.-Обоснуйте,-почему-качество-линейной-регрессии-не-поменяется." data-toc-modified-id="Предложите-алгоритм-преобразования-данных-для-решения-задачи.-Обоснуйте,-почему-качество-линейной-регрессии-не-поменяется.-3.1"><span class="toc-item-num">3.1&nbsp;&nbsp;</span>Предложите алгоритм преобразования данных для решения задачи. Обоснуйте, почему качество линейной регрессии не поменяется.</a></span></li></ul></li><li><span><a href="#Проверка-алгоритма" data-toc-modified-id="Проверка-алгоритма-4"><span class="toc-item-num">4&nbsp;&nbsp;</span>Проверка алгоритма</a></span></li><li><span><a href="#Чек-лист-проверки" data-toc-modified-id="Чек-лист-проверки-5"><span class="toc-item-num">5&nbsp;&nbsp;</span>Чек-лист проверки</a></span></li></ul></div>

In [1]:
# Подключим необходимые библиотеки
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
import seaborn as sb
import math
# Для визуализации формулы
# import latexify

from sklearn.metrics import r2_score, mean_squared_error

from sklearn.linear_model import LinearRegression

## Загрузка данных

In [2]:
try:
    # Локальный файл
    try:
        df = pd.read_csv('datasets/insurance.csv')
    # Файл с Яндекса
    except:
        df = pd.read_csv('/datasets/insurance.csv')
except:
    print('Не найден датасет. Проверьте адрес указанного файла')

In [3]:
df.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 5000 entries, 0 to 4999
Data columns (total 5 columns):
 #   Column             Non-Null Count  Dtype  
---  ------             --------------  -----  
 0   Пол                5000 non-null   int64  
 1   Возраст            5000 non-null   float64
 2   Зарплата           5000 non-null   float64
 3   Члены семьи        5000 non-null   int64  
 4   Страховые выплаты  5000 non-null   int64  
dtypes: float64(2), int64(3)
memory usage: 195.4 KB


In [4]:
df.describe()

,Пол,Возраст,Зарплата,Члены семьи,Страховые выплаты
count,5000.000000,5000.000000,5000.000000,5000.000000,5000.000000
mean,0.499000,30.952800,39916.360000,1.194200,0.148000
std,0.500049,8.440807,9900.083569,1.091387,0.463183
min,0.000000,18.000000,5300.000000,0.000000,0.000000
25%,0.000000,24.000000,33300.000000,0.000000,0.000000
50%,0.000000,30.000000,40200.000000,1.000000,0.000000
75%,1.000000,37.000000,46600.000000,2.000000,0.000000
max,1.000000,65.000000,79000.000000,6.000000,5.000000


Здесь невооруженным глазом видно, что отсутствуют аномальные значения. Проверяем на пропущенные значения

In [5]:
df.isna().sum()

Пол                  0
Возраст              0
Зарплата             0
Члены семьи          0
Страховые выплаты    0
dtype: int64

Отсутствуют пропущенные значения

## Умножение матриц

In [6]:
# Создадим класс
class My_class_LinearRegression:
    def fit(self, train_features, train_target):
        X = np.concatenate((np.ones((train_features.shape[0], 1)), train_features), axis=1)
        y = train_target
        w = np.dot(np.dot(np.linalg.inv(np.dot(X.T, X)), X.T), y)
        self.w = w[1:]
        self.w0 = w[0]

    def predict(self, test_features):
        return test_features.dot(self.w) + self.w0

In [7]:
# Признаки
features = df.drop('Страховые выплаты', axis=1)
# Целевой признак
target = df['Страховые выплаты']

In [8]:
model_my_class_lin_reg = My_class_LinearRegression()
model_my_class_lin_reg.fit(features, target)

In [9]:
predictions = model_my_class_lin_reg.predict(features)
print('Предсказания:', predictions.mean())

Предсказания: 0.148


In [10]:
model_my_class_lin_reg.w

array([ 7.92580543e-03,  3.57083050e-02, -1.70080492e-07, -1.35676623e-02])

In [11]:
print('MSE:', mean_squared_error(target, predictions))

MSE: 0.1233468894171086


In [12]:
print('R2_score:', r2_score(target, predictions))

R2_score: 0.42494550286668


In [13]:
model_lin_reg = LinearRegression()
model_lin_reg.fit(features, target)

LinearRegression()

In [14]:
predictions = model_lin_reg.predict(features)
print('Предсказания:', predictions.mean())

Предсказания: 0.14799999999999996


In [15]:
print('MSE:', mean_squared_error(target, predictions))

MSE: 0.12334688941710859


In [16]:
print('R2_score:', r2_score(target, predictions))

R2_score: 0.4249455028666801


In [17]:
model_lin_reg.coef_

array([ 7.92580543e-03,  3.57083050e-02, -1.70080492e-07, -1.35676623e-02])

In [18]:
# Проверим на размерность
print(len(model_my_class_lin_reg.w) == len(model_lin_reg.coef_))

print(model_my_class_lin_reg.w == model_lin_reg.coef_)

True
[False False False False]


Как то странно, видно, что числа одинаковые, но компилятор говорит, что не равны. Если не будем мелочиться это, то получается, что по требуемой формулой мы добились такого же результата, что и как у Линейней регрессией

В этом задании вы можете записывать формулы в *Jupyter Notebook.*

Чтобы записать формулу внутри текста, окружите её символами доллара \\$; если снаружи —  двойными символами \\$\\$. Эти формулы записываются на языке вёрстки *LaTeX.*

Для примера мы записали формулы линейной регрессии. Можете их скопировать и отредактировать, чтобы решить задачу.

Работать в *LaTeX* необязательно.

Обозначения:

- $X$ — матрица признаков (нулевой столбец состоит из единиц)

- $y$ — вектор целевого признака

- $P$ — матрица, на которую умножаются признаки

- $w$ — вектор весов линейной регрессии (нулевой элемент равен сдвигу)

в Предсказания:

$$
a = Xw
$$

Задача обучения:

$$
w = \arg\min_w MSE(Xw, y)
$$

Формула обучения:

$$
w = (X^T X)^{-1} X^T y
$$

**Вопрос:** Признаки умножают на обратимую матрицу? Изменится ли качество линейной регрессии?

**Ответ:**

Нет, не изменится качество линейной регрессии после умножения на обратную матрицы

**Обоснование:**

Немного об обратимой матрицы.
Квадратные матрицы, для которых можно найти обратные, называются **обратимыми** (англ. invertible matrix). Но не у каждой матрицы есть обратная. Например, у матрицы A, состоящей только из нулей, нет обратной: в результате любого умножения будут только нули. Такая матрица называется **необратимой** (англ. non-invertible matrix).
**Обратная для квадратной матрицы A** (англ. inverse matrix) — матрица A с верхним индексом -1, произведение которой на А равно единичной матрице. Умножение может быть в любом порядке:
$$ AA^{-1} = A^{-1}A = E $$

Пусть F (Features) произвольная матрица признаков, размерность которой равна размерности матрицы признаков X, тогда: $ F \equiv X $ (эквивалентны)

_*Примечание: Матрица признака должна быть квадратичной формой, т.е. длина строк равна длине столбцов*_ $ i = j $

Тогда у нас в этом случае можно рассмотреть так:

$$ F = XP $$

где:
    - $X$ - матрица признаков (нулевой столбец состоит из единиц)
    - $P$ — матрица, на которую умножаются признаки


Формула предсказания будет иметь следующий вид:
$$ a = Fw $$

Вектор весов линейной регрессии:
$$ w = \frac{a}{F} $$

Формула обучения:
$$ w = (F^T  F)^{-1} F^T y $$

Подставляем формулу предсказания в формулу обучения:
$$ \frac{a}{F} = (F^T  F)^{-1} X^T y $$

Умножим признаки F на обе стороны, чтобы перенести признак на правую часть
$$ a = F(F^T  F)^{-1} X^T y $$

Значения $ F = XP $ подставляем в эту формулу $ a = F(F^T  F)^{-1} X^T y $ и получим:

$$ a = XP((XP)^T XP)^{-1} (XP)^T y$$

$$ / (XP)^T = X^T P^T / $$

$$ a = XP(X^T P^T)^{-1} X^{-1} P^{-1} X^T P^T y$$

$$ a = XX^{-1} PP^{-1} (X^T P^T)^{-1}  X^T P^T y$$

$$ / PP^{-1} = E / $$

$$ / (X^T P^T)^{-1} = (X^T)^{-1} (P^T)^{-1} / $$

$$ a = XX^{-1} E (X^T)^{-1} (P^T)^{-1} X^T P^T y$$

$$ a = XX^{-1} E (X^T)^{-1}X^T (P^T)^{-1} P^T y$$

$$ / (P^T)^{-1} P^T = E / $$

$$ a = XX^{-1} E (X^T)^{-1}X^T E y$$

$$ / E*E = E / $$

$$ a = XX^{-1} (X^T)^{-1}X^T y E$$

$$ / X^{-1} (X^T)^{-1} = (XX^T)^{-1} / $$

$$ a = X (XX^T)^{-1} X^T y E$$

$$ По \quad формуле \quad обучения: w = (X^T X)^{-1} X^T y $$

$$ a = X w E = X w $$



Как видим, что после умножение матрицы признаков на обратимую матрицу предсказания a не изменилась. Следовательно, качество линейной регрессии не изменяется, т.к. она не зависит от масштаба признаков. Если хотим улучшить качество линейной регрессии, тогда нужно преобразовать признаки или добавить новые признаки

## Алгоритм преобразования

### Предложите алгоритм преобразования данных для решения задачи. Обоснуйте, почему качество линейной регрессии не поменяется.

**Алгоритм**

Нам нужно разработать алгоритм преобразования данных, чтобы по ним было сложно восстановить персональную информацию. Но при этом нужно защитить данные, чтобы при преобразовании качество моделей машинного обучения не ухудшилось.

Тогда здесь алгоритм будет:
1. Произвольно создадим матрицу признаков, размер которой равен размеру матрицы признаков (в этом случае произвольно создадим квадратную матрицу)
2. Умножим эти матрицы
3. Добавить нулевой столбец в полученном матрице и выводить в линейную регрессию

Следовательно:
$$ F = XA $$

где:
    - $X$ - матрица признаков
    - $A$ — произвольная матрица, на которую умножаются признаки

**Обоснование**

Пусть произвольная (A - arbitrary) матрица будет иметь следующий вид:
$$
A =
 \begin{pmatrix}
  a_{1,1} & a_{1,2} & \cdots & a_{1,n} \\
  a_{2,1} & a_{2,2} & \cdots & a_{2,n} \\
  \vdots  & \vdots  & \ddots & \vdots  \\
  a_{n,1} & a_{n,2} & \cdots & a_{n,n}
 \end{pmatrix}
$$

А матрица признаков:

$$
X =
 \begin{pmatrix}
  x_{1,1} & x_{1,2} & \cdots & x_{1,n} \\
  x_{2,1} & x_{2,2} & \cdots & x_{2,n} \\
  \vdots  & \vdots  & \ddots & \vdots  \\
  x_{m,1} & x_{m,2} & \cdots & x_{m,n}
 \end{pmatrix}
$$

_*Примечание: кол-во столбцов матрицы признаков должны быть равен кол-ву строк произвольной матрицы*_

Тогда полученная матрица будет:

$$
F =
 \begin{pmatrix}
  f_{1,1} & f_{1,2} & \cdots & f_{1,n} \\
  f_{2,1} & f_{2,2} & \cdots & f_{2,n} \\
  \vdots  & \vdots  & \ddots & \vdots  \\
  f_{m,1} & f_{m,2} & \cdots & f_{m,n}
 \end{pmatrix}
$$

## Проверка алгоритма

Нам известно, что приблизительное значение R2_score и MSE выдали: 0,42 и 0,12. Теперь преобразуем матрицу и смотрим, сильно ли изменились эти значения?

In [19]:
# Создадим произвольную квадратную матрицу (определитель не должна быть равна нулю, т.е. матрица должна быть невырожденной)
while(True):
    sqaure_mathric = np.random.randint(1, 10, (features.shape[1], features.shape[1]))
    print('DET:', np.linalg.det(sqaure_mathric))
    if(np.linalg.det(sqaure_mathric) != 0): break

DET: 295.99999999999994


In [20]:
# Новый признак
new_features = np.dot(features, sqaure_mathric)

Проведем сравнения моделей до и после преобразования

In [21]:
# До преобразования
features

,Пол,Возраст,Зарплата,Члены семьи
0,1,41.0,49600.0,1
1,0,46.0,38000.0,1
2,0,29.0,21000.0,0
3,0,21.0,41700.0,2
4,1,28.0,26100.0,0
...,...,...,...,...
4995,0,28.0,35700.0,2
4996,0,34.0,52400.0,1
4997,0,20.0,33900.0,2
4998,1,22.0,32700.0,3


In [22]:
# После преобразования
new_features

array([[397133.,  99420., 198658., 297736.],
       [304371.,  76238., 152283., 228142.],
       [168232.,  42145.,  84174., 126087.],
       ...,
       [271366.,  67916., 135734., 203468.],
       [261787.,  65541., 130958., 196287.],
       [325029.,  81355., 162580., 243697.]])

Как и видим, что здесь присутствуют постронние числа. Т.е. это условно можно назвать как шифрованием. Теперь проверим на метрику R2_score и MSE

In [23]:
model_lin_reg.fit(new_features, target)

LinearRegression()

In [24]:
predictions = model_lin_reg.predict(new_features)
print('Предсказания:', predictions.mean())

Предсказания: 0.14800000000044472


In [25]:
print('MSE:', mean_squared_error(target, predictions))

MSE: 0.12334688941710142


In [26]:
print('R2_score:', r2_score(target, predictions))

R2_score: 0.4249455028667135


Как и видим, что ничего не поменялось

Вывод:
Мы выяснили, что качество линейной регрессии не изменилось после умножения на обратной матрицы. Что и теперь можем защищать информацию, применяя знания линейной алгебры - преобразовав матрицу (шифровка), и при этом можем сохранить значения r2_score и mse